<a href="https://colab.research.google.com/github/SubramanyaJ/23CS5BSBIS/blob/main/Lab_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random

def initialize_pheromone(num_cities, initial_pheromone=1.0):
    return np.ones((num_cities, num_cities)) * initial_pheromone

def calculate_probabilities(pheromone, distances, visited, alpha=1, beta=2):
    pheromone = np.copy(pheromone)

    prob_matrix = np.zeros_like(pheromone)
    for i in range(len(pheromone)): # Iterate over cities in the pheromone row
        if i not in visited:
            prob_matrix[i] = pheromone[i]

    heuristic = 1 / (distances + 1e-10)
    for i in range(len(distances)):
        if i in visited:
            heuristic[i] = 0

    prob = (prob_matrix ** alpha) * (heuristic ** beta)
    total = np.sum(prob)
    if total == 0:
        choices = [i for i in range(len(distances)) if i not in visited]
        return choices, None
    prob = prob / total
    return range(len(distances)), prob

def select_next_city(probabilities, cities):
    if probabilities is None:
        return random.choice(cities)
    return np.random.choice(cities, p=probabilities)

def path_cost(path, distances, times):
    total_distance = 0
    total_time = 0

    for i in range(len(path)):
        total_distance += distances[path[i-1]][path[i]]
        total_time += times[path[i-1]][path[i]]
    return total_distance, total_time

def ant_colony_optimization(distances, times, time_window, n_ants=5, n_iterations=50, rho=0.5, alpha=1, beta=3):
    num_cities = len(distances)
    pheromone = initialize_pheromone(num_cities)
    best_path = None
    best_distance = float('inf')
    best_time = float('inf')

    for iteration in range(n_iterations):
        all_paths_data = []
        for _ in range(n_ants):
            path = [0]
            visited = set(path)
            current_path_time = 0.0
            current_path_distance = 0.0
            path_incomplete = False

            for step in range(num_cities - 1):
                current_city = path[-1]

                unvisited_cities = [c for c in range(num_cities) if c not in visited]

                valid_next_step_candidates = []
                for next_city_candidate in unvisited_cities:
                    time_to_next_leg = times[current_city][next_city_candidate]

                    # updation check
                    if current_path_time + time_to_next_leg <= time_window:
                        valid_next_step_candidates.append(next_city_candidate)

                if not valid_next_step_candidates:
                    path_incomplete = True
                    break

                temp_current_pheromone = pheromone[current_city].copy()
                temp_current_distances = distances[current_city].copy()

                for c in range(num_cities):
                    if c not in valid_next_step_candidates or c in visited:
                        temp_current_pheromone[c] = 0
                        temp_current_distances[c] = np.inf

                cities_indices, probabilities_array = calculate_probabilities(
                    temp_current_pheromone,
                    temp_current_distances,
                    visited,
                    alpha, beta
                )

                if probabilities_array is None or np.sum(probabilities_array) == 0:
                    if not valid_next_step_candidates:
                        path_incomplete = True
                        break
                    next_city = random.choice(valid_next_step_candidates)
                else:
                    next_city = np.random.choice(cities_indices, p=probabilities_array / np.sum(probabilities_array))

                path.append(next_city)
                visited.add(next_city)
                current_path_distance += distances[current_city][next_city]
                current_path_time += times[current_city][next_city]

            if not path_incomplete and len(path) == num_cities:
                total_dist, total_tm = path_cost(path, distances, times)

                if total_tm <= time_window:
                    all_paths_data.append((path, total_dist, total_tm))

                    if total_dist < best_distance:
                        best_distance = total_dist
                        best_path = path
                        best_time = total_tm

        pheromone *= (1 - rho)

        for path_found, dist, tm in all_paths_data:
            deposit = 1.0 / dist
            for i in range(len(path_found)):
                city_from = path_found[i-1]
                city_to = path_found[i]
                pheromone[city_from][city_to] += deposit

    return best_path, best_distance, best_time

# main
distances = np.array(
    [
        [np.inf, 2, 2, 5, 7],
        [2, np.inf, 4, 8, 2],
        [2, 4, np.inf, 1, 3],
        [5, 8, 1, np.inf, 2],
        [7, 2, 3, 2, np.inf]
    ]
  )

times = np.array(
    [
        [np.inf, 10, 15, 20, 25],
        [10, np.inf, 12, 18, 14],
        [15, 12, np.inf, 8, 10],
        [20, 18, 8, np.inf, 5],
        [25, 14, 10, 5, np.inf]
    ]
)

time_window = 52

best_path, best_dist, best_time = ant_colony_optimization(distances, times, time_window)
if best_path is not None:
    print(f"Best path: {[int(city) for city in best_path]}\nDistance: {best_dist}\nTime: {best_time}")
else:
    print("No valid path found within the time window.")


Best path: [0, 2, 3, 4, 1]
Distance: 9.0
Time: 52.0
